In [89]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string
import nltk
from datetime import timedelta, date
import json
from ast import literal_eval
import ast
import requests
from bs4 import BeautifulSoup

In [90]:
file_path = "Data/natgeo.csv"
df = pd.read_csv(file_path)

Kiểm tra xem yêu cầu về thời gian các bài đăng crawl được có đạt tiêu chuẩn ít nhất 1 tháng không, xem có dữ liệu nào bị lỗi cần sửa không và lưu các thông tin cần dùng vào file csv riêng

In [91]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 230 entries, 0 to 229
Data columns (total 51 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   post_id                        230 non-null    int64  
 1   text                           230 non-null    object 
 2   post_text                      230 non-null    object 
 3   shared_text                    150 non-null    object 
 4   original_text                  0 non-null      float64
 5   time                           230 non-null    object 
 6   timestamp                      230 non-null    int64  
 7   image                          47 non-null     object 
 8   image_lowquality               230 non-null    object 
 9   images                         230 non-null    object 
 10  images_description             230 non-null    object 
 11  images_lowquality              230 non-null    object 
 12  images_lowquality_description  230 non-null    obj

In [45]:
df['time'] = pd.to_datetime(df['time'])

ValueError: unconverted data remains when parsing with format "%Y-%m-%d %H:%M:%S": " ", at position 50. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [92]:
df['time'] = pd.to_datetime(df['time'].str.strip())

In [93]:
df['day'] = df['time'].dt.day
print(df['day'].nunique())

30


In [94]:
columns =["time","likes","comments","reactions","reaction_count","image","video","link","text",'shares','post_url','comments_full']
select = df[columns]

In [138]:
a = select['link'].notna().info()

<class 'pandas.core.series.Series'>
RangeIndex: 230 entries, 0 to 229
Series name: link
Non-Null Count  Dtype
--------------  -----
230 non-null    bool 
dtypes: bool(1)
memory usage: 362.0 bytes


In [96]:
select['topic']= None

C:\Users\ACER\AppData\Local\Temp\ipykernel_20612\4201195324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['topic']= None


In [97]:
def function(value):
    if(pd.notna(value)):
        response = requests.get(value)
        bs = BeautifulSoup(response.text,"lxml")
        if(bs.find("li",{"class":"SectionLabel SectionLabel--link"})):
            return bs.find("li",{"class":"SectionLabel SectionLabel--link"}).text
        elif(bs.find("div",{"class":"LeadContent__SectionLabels"})):
            return bs.find("div",{"class":"LeadContent__SectionLabels"}).text
        else:
            return "Special"
    else:
        return None

In [98]:
select['topic'] = select['link'].apply(function)

C:\Users\ACER\AppData\Local\Temp\ipykernel_20612\4224345657.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  select['topic'] = select['link'].apply(function)


In [104]:
a = select[select['topic'].isna()]
print(len(a))

59


In [103]:
a = select[select['topic'].notna()]
print(len(a))

171


In [101]:
a = select[select['topic']=="Special"]
for index,row in a.iterrows():
    print(index, row['link'])

119 https://on.natgeo.com/3FVNEDA?fbclid=IwAR1fi4XD9XJ181N2J4IFwR1nbrBCn_UN8Ay7j-o21lbFGxv-BZwaFdFR1o0
120 https://on.natgeo.com/3ugyNRy?fbclid=IwAR20aKfPWvuR9almx7wpA1uOz2buN06g5dEDNTEH_WZeVxU30msiXVJ9Uw8
134 https://on.natgeo.com/46bXhbI?fbclid=IwAR29gBpWzFt-YnOmkdD6pje85IHi_n4EkWmAODugN963BrDc9OPy86_6Yy4


In [102]:
select.at[119,"topic"] = "Photography"
select.at[120,"topic"] = "Environment"
select.at[134,"topic"] = "History & Culture"

In [107]:
value_counts = select['topic'].value_counts()

# Convert the result to a list of tuples (value, count)
value_counts_list = value_counts.reset_index().values.tolist()
value_counts_list

[['Travel', 53],
 ['Science', 34],
 ['History & Culture', 30],
 ['Animals', 25],
 ['Environment', 13],
 ['Photography', 8],
 ['History Magazine', 6],
 ['Magazine', 2]]

In [140]:
topic_data = select[(select['topic'] != "History Magazine") & (select['topic']!= "Magazine")]

In [141]:
topic_data_nona = topic_data[topic_data['topic'].notna()]
topic_data_nona.shape[0]

163

In [142]:
topic_data_na = topic_data[topic_data['topic'].isna()]
topic_data_na.shape[0]

59

In [145]:
value_counts = topic_data['topic'].value_counts()

count_df = pd.DataFrame({'topic': value_counts.index, 'Count': value_counts.values})
count_df

,topic,Count
0,Travel,53
1,Science,34
2,History & Culture,30
3,Animals,25
4,Environment,13
5,Photography,8


In [116]:
# Tải các tài nguyên của nltk
nltk.download('stopwords')
nltk.download('punkt')

# dữ liệu train thuật toán

df_training = topic_data_nona

# Tokenization, stopword removal, and stemming
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def process_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Áp dụng hàm xử lý văn bản cho cột text
df_training['processed_text'] = df_training['text'].apply(process_text)

# Tách dữ liệu văn bản qua xử lý thành hai tập, tập train và tập test
train_data, test_data, train_labels, test_labels = train_test_split(
    df_training['processed_text'], df_training['topic'], test_size=0.3, random_state=42
)

# vecto hóa văn bản qua xử lý với TF-IDF
vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(train_data)
test_vectors = vectorizer.transform(test_data)
class_priors = {'History & Culture': 0.8, 'Travel': 0.9, 'Science': 1.0,'Animals':1.0,'Photography':1.0,'Environment':1.0}
class_priors_normalized = [class_priors[label] / sum(class_priors.values()) for label in train_labels.unique()]
# Train một Multinomial Naive Bayes classifier
classifier = MultinomialNB(class_prior=class_priors_normalized)
classifier.fit(train_vectors, train_labels)

train_predictions = classifier.predict(train_vectors)

# Tính toán và xét dộ chính xác của thuật toán bằng cách thử vào dữ liệu đã có topic
train_accuracy = accuracy_score(train_labels, train_predictions)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

#các bài đăng cần topic
df_unlabeled = topic_data_na  

# xử lý văn bản
df_unlabeled['processed_text'] = df_unlabeled['text'].apply(process_text)

# vecto hóa văn bản được xử lý
unlabeled_vectors = vectorizer.transform(df_unlabeled['processed_text'])

# Đoán các topic phù hợp cho văn bản
predicted_categories = classifier.predict(unlabeled_vectors)

# lưu các topic dự doán
df_unlabeled['PredictedTopic'] = predicted_categories


Training Accuracy: 100.00%


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
C:\Users\ACER\AppData\Local\Temp\ipykernel_20612\2025032158.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_training['processed_text'] = df_training['text'].apply(process_text)
C:\Users\ACER\AppData\Local\Temp\ipykernel_20612\2025032158.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [117]:
df_unlabeled['topic'] = df_unlabeled['PredictedTopic']

C:\Users\ACER\AppData\Local\Temp\ipykernel_20612\25949949.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unlabeled['topic'] = df_unlabeled['PredictedTopic']


In [118]:
value_counts = df_unlabeled['PredictedTopic'].value_counts()

# Convert the result to a list of tuples (value, count)
value_counts_list = value_counts.reset_index().values.tolist()
value_counts_list

[['History & Culture', 17],
 ['Travel', 15],
 ['Animals', 14],
 ['Environment', 8],
 ['Photography', 4],
 ['Science', 1]]

In [119]:
df_labeled = pd.concat([df_training, df_unlabeled], ignore_index=True)
df_labeled

,time,likes,comments,reactions,reaction_count,image,video,link,text,shares,post_url,topic,processed_text,PredictedTopic
0,2023-11-30 21:15:00,383,6,"{'like': 383, 'love': 11, 'care': 2, 'haha':1,...",397,NaN,NaN,https://www.nationalgeographic.com/history/art...,How we've dealt with periods over millennia sa...,15,https://facebook.com/natgeo/posts/897234058440692,History & Culture,dealt period millennia say lot persist stigma ...,NaN
1,2023-11-30 18:15:00,675,20,"{'like': 675, 'love': 46, 'care': 4, 'haha':2,...",730,NaN,NaN,https://www.nationalgeographic.com/environment...,What will it take the save the world's rivers?...,35,https://facebook.com/natgeo/posts/897234058440692,Environment,take save world river expert say need look vjo...,NaN
2,2023-11-30 12:15:00,710,12,"{'like': 710, 'love': 41, 'care': 0, 'haha':2,...",754,NaN,NaN,https://www.nationalgeographic.com/travel/arti...,North America is becoming a more popular winte...,15,https://facebook.com/natgeo/posts/897234058440692,Travel,north america becom popular winter sport desti...,NaN
3,2023-11-30 09:15:00,501,22,"{'like': 501, 'love': 46, 'care': 5, 'haha':1,...",554,NaN,NaN,https://www.nationalgeographic.com/premium/art...,Experts have one key piece of advice for our a...,68,https://facebook.com/natgeo/posts/897234058440692,Science,expert one key piec advic age bodi stop move w...,NaN
4,2023-11-30 04:15:00,564,13,"{'like': 564, 'love': 36, 'care': 10, 'haha':0...",618,NaN,NaN,https://www.nationalgeographic.com/animals/art...,Can the black hairs of this woolly bear caterp...,15,https://facebook.com/natgeo/posts/897234058440692,Animals,black hair woolli bear caterpillar realli tell...,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,2023-11-02 23:18:00,1000,28,"{'like': 1000, 'love': 149, 'care': 18, 'haha'...",1200,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,NaN,NaN,"At a Day of the Dead celebration in Hopelchen,...",106,https://facebook.com/natgeo/posts/897234058440692,History & Culture,day dead celebr hopelchen mexico famili offer ...,History & Culture
218,2023-11-02 05:15:00,1300,36,"{'like': 1300, 'love': 258, 'care': 28, 'haha'...",1600,NaN,https://scontent.fhan14-2.fna.fbcdn.net/v/t42....,NaN,Did you know monarchs are a spiritual symbol f...,242,https://facebook.com/natgeo/posts/897234058440692,Photography,know monarch spiritu symbol day dead celebr na...,Photography
219,2023-11-01 23:15:00,1900,8,"{'like': 1900, 'love': 181, 'care': 5, 'haha':...",2200,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,NaN,NaN,Mountain birch and Scots pine forests are dist...,86,https://facebook.com/natgeo/posts/897234058440692,Travel,mountain birch scot pine forest distinctli sep...,Travel
220,2023-11-01 17:15:00,334,21,"{'like': 334, 'love': 14, 'care': 10, 'haha':8...",382,NaN,https://scontent.fhan14-2.fna.fbcdn.net/v/t42....,NaN,Have you heard of delusional parasitosis? Some...,14,https://facebook.com/natgeo/posts/897234058440692,Photography,heard delusion parasitosi peopl believ tini bu...,Photography


In [120]:
done = df_labeled.drop('PredictedTopic',axis=1)
done

,time,likes,comments,reactions,reaction_count,image,video,link,text,shares,post_url,topic,processed_text
0,2023-11-30 21:15:00,383,6,"{'like': 383, 'love': 11, 'care': 2, 'haha':1,...",397,NaN,NaN,https://www.nationalgeographic.com/history/art...,How we've dealt with periods over millennia sa...,15,https://facebook.com/natgeo/posts/897234058440692,History & Culture,dealt period millennia say lot persist stigma ...
1,2023-11-30 18:15:00,675,20,"{'like': 675, 'love': 46, 'care': 4, 'haha':2,...",730,NaN,NaN,https://www.nationalgeographic.com/environment...,What will it take the save the world's rivers?...,35,https://facebook.com/natgeo/posts/897234058440692,Environment,take save world river expert say need look vjo...
2,2023-11-30 12:15:00,710,12,"{'like': 710, 'love': 41, 'care': 0, 'haha':2,...",754,NaN,NaN,https://www.nationalgeographic.com/travel/arti...,North America is becoming a more popular winte...,15,https://facebook.com/natgeo/posts/897234058440692,Travel,north america becom popular winter sport desti...
3,2023-11-30 09:15:00,501,22,"{'like': 501, 'love': 46, 'care': 5, 'haha':1,...",554,NaN,NaN,https://www.nationalgeographic.com/premium/art...,Experts have one key piece of advice for our a...,68,https://facebook.com/natgeo/posts/897234058440692,Science,expert one key piec advic age bodi stop move w...
4,2023-11-30 04:15:00,564,13,"{'like': 564, 'love': 36, 'care': 10, 'haha':0...",618,NaN,NaN,https://www.nationalgeographic.com/animals/art...,Can the black hairs of this woolly bear caterp...,15,https://facebook.com/natgeo/posts/897234058440692,Animals,black hair woolli bear caterpillar realli tell...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,2023-11-02 23:18:00,1000,28,"{'like': 1000, 'love': 149, 'care': 18, 'haha'...",1200,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,NaN,NaN,"At a Day of the Dead celebration in Hopelchen,...",106,https://facebook.com/natgeo/posts/897234058440692,History & Culture,day dead celebr hopelchen mexico famili offer ...
218,2023-11-02 05:15:00,1300,36,"{'like': 1300, 'love': 258, 'care': 28, 'haha'...",1600,NaN,https://scontent.fhan14-2.fna.fbcdn.net/v/t42....,NaN,Did you know monarchs are a spiritual symbol f...,242,https://facebook.com/natgeo/posts/897234058440692,Photography,know monarch spiritu symbol day dead celebr na...
219,2023-11-01 23:15:00,1900,8,"{'like': 1900, 'love': 181, 'care': 5, 'haha':...",2200,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,NaN,NaN,Mountain birch and Scots pine forests are dist...,86,https://facebook.com/natgeo/posts/897234058440692,Travel,mountain birch scot pine forest distinctli sep...
220,2023-11-01 17:15:00,334,21,"{'like': 334, 'love': 14, 'care': 10, 'haha':8...",382,NaN,https://scontent.fhan14-2.fna.fbcdn.net/v/t42....,NaN,Have you heard of delusional parasitosis? Some...,14,https://facebook.com/natgeo/posts/897234058440692,Photography,heard delusion parasitosi peopl believ tini bu...


In [126]:
done['topic'].value_counts()

topic
Travel               68
History & Culture    47
Animals              39
Science              35
Environment          21
Photography          12
Name: count, dtype: int64

In [127]:
a = select[(select['topic'] == "History Magazine") | (select['topic']== "Magazine")]
a

,time,likes,comments,reactions,reaction_count,image,video,link,text,shares,post_url,topic
18,2023-11-28 12:15:00,1000,28,"{'like': 1000, 'love': 72, 'care': 4, 'haha':2...",1100,NaN,NaN,https://www.nationalgeographic.com/history/his...,"Discovered in the 1970s, the terra-cotta army ...",56,https://facebook.com/natgeo/posts/897234058440692,History Magazine
74,2023-11-21 02:15:03,308,11,"{'like': 308, 'love': 30, 'wow': 4, 'care': 3,...",346,NaN,NaN,https://on.natgeo.com/47IngJk?fbclid=IwAR00cTf...,The Princes in the Tower went missing 540 year...,8,https://facebook.com/natgeo/posts/895895755241189,History Magazine
102,2023-11-17 06:22:15,564,14,"{'like': 564, 'love': 109, 'haha': 3, 'wow': 3...",690,NaN,https://scontent.fhan14-3.fna.fbcdn.net/v/t42....,https://on.natgeo.com/40JjMnl?fbclid=IwAR0QYJn...,"Across the continent, Indigenous peoples are m...",65,https://facebook.com/natgeo/posts/893743538789744,Magazine
104,2023-11-17 02:15:05,635,19,"{'like': 635, 'love': 15, 'wow': 2, 'sad': 1}",653,NaN,NaN,https://on.natgeo.com/3sLSAYu?fbclid=IwAR0olR9...,"Despite being written for his eyes only, the p...",53,https://facebook.com/natgeo/posts/893665255464239,History Magazine
128,2023-11-14 04:15:05,537,16,"{'like': 537, 'love': 99, 'haha': 2, 'wow': 1,...",656,NaN,NaN,https://on.natgeo.com/40EdjtK?fbclid=IwAR2D4lr...,All over Turtle Island—a common Indigenous nam...,28,https://facebook.com/natgeo/posts/892111482286283,Magazine
193,2023-11-05 11:15:00,978,38,"{'like': 978, 'love': 22, 'care': 6, 'haha':2,...",1000,NaN,NaN,https://www.nationalgeographic.com/premium/art...,"Odysseus was a mythological character, but the...",62,https://facebook.com/natgeo/posts/897234058440692,History Magazine
207,2023-11-03 17:15:00,315,14,"{'like': 315, 'love': 13, 'care': 1, 'haha':6,...",339,NaN,NaN,https://www.nationalgeographic.com/premium/art...,"King Louis XIV, famous for his ostentatious se...",17,https://facebook.com/natgeo/posts/897234058440692,History Magazine
211,2023-11-03 03:15:00,939,70,"{'like': 939, 'love': 22, 'care': 7, 'haha':1,...",1100,https://scontent.fhan14-4.fna.fbcdn.net/v/t39....,NaN,https://on.natgeo.com/3u4GRo9,We thought we knew the secrets of Europe's bog...,49,https://facebook.com/natgeo/posts/897234058440692,History Magazine


In [128]:
result = pd.concat([done, a], ignore_index=True)
result['topic'].value_counts()

topic
Travel               68
History & Culture    47
Animals              39
Science              35
Environment          21
Photography          12
History Magazine      6
Magazine              2
Name: count, dtype: int64

In [131]:
final = result.sort_values(by='time', ascending=False)
final

,time,likes,comments,reactions,reaction_count,image,video,link,text,shares,post_url,topic,processed_text
0,2023-11-30 21:15:00,383,6,"{'like': 383, 'love': 11, 'care': 2, 'haha':1,...",397,NaN,NaN,https://www.nationalgeographic.com/history/art...,How we've dealt with periods over millennia sa...,15,https://facebook.com/natgeo/posts/897234058440692,History & Culture,dealt period millennia say lot persist stigma ...
1,2023-11-30 18:15:00,675,20,"{'like': 675, 'love': 46, 'care': 4, 'haha':2,...",730,NaN,NaN,https://www.nationalgeographic.com/environment...,What will it take the save the world's rivers?...,35,https://facebook.com/natgeo/posts/897234058440692,Environment,take save world river expert say need look vjo...
2,2023-11-30 12:15:00,710,12,"{'like': 710, 'love': 41, 'care': 0, 'haha':2,...",754,NaN,NaN,https://www.nationalgeographic.com/travel/arti...,North America is becoming a more popular winte...,15,https://facebook.com/natgeo/posts/897234058440692,Travel,north america becom popular winter sport desti...
3,2023-11-30 09:15:00,501,22,"{'like': 501, 'love': 46, 'care': 5, 'haha':1,...",554,NaN,NaN,https://www.nationalgeographic.com/premium/art...,Experts have one key piece of advice for our a...,68,https://facebook.com/natgeo/posts/897234058440692,Science,expert one key piec advic age bodi stop move w...
163,2023-11-30 06:15:00,6800,357,"{'like': 6800, 'love': 2100, 'care': 107, 'hah...",9400,NaN,https://scontent.fhan14-2.fna.fbcdn.net/v/t42....,NaN,A traffic cam along I-94 in Minnesota captured...,1000,https://facebook.com/natgeo/posts/897234058440692,Animals,traffic cam along minnesota captur fine footag...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,2023-11-01 11:15:00,1000,14,"{'like': 1000, 'love': 67, 'care': 10, 'haha':...",1000,NaN,NaN,https://www.nationalgeographic.com/travel/arti...,The Swedish capital's character has been shape...,29,https://facebook.com/natgeo/posts/897234058440692,Travel,swedish capit charact shape water life scatter...
157,2023-11-01 08:15:00,416,13,"{'like': 416, 'love': 11, 'care': 1, 'haha':0,...",429,NaN,NaN,https://www.nationalgeographic.com/premium/art...,"When certain vitamins, minerals, or phytonutri...",34,https://facebook.com/natgeo/posts/897234058440692,Science,certain vitamin miner phytonutri consum togeth...
158,2023-11-01 08:15:00,416,13,"{'like': 416, 'love': 11, 'care': 1, 'haha':0,...",429,NaN,NaN,https://www.nationalgeographic.com/premium/art...,"When certain vitamins, minerals, or phytonutri...",34,https://facebook.com/natgeo/posts/897234058440692,Science,certain vitamin miner phytonutri consum togeth...
162,2023-11-01 03:15:00,447,44,"{'like': 447, 'love': 7, 'care': 5, 'haha':3, ...",513,NaN,NaN,https://www.nationalgeographic.com/history/art...,"The backstory of jack-o'-lanterns, including h...",61,https://facebook.com/natgeo/posts/897234058440692,History & Culture,backstori includ came star halloween decor car...


In [135]:
final['processed_text'] = final['text'].apply(process_text)
final['engagement'] = final['shares'] + final['reaction_count'] +final['comments'] 

In [136]:
final.to_csv("Data/final.csv",index=False)